# 0.6 - Agente com Memória de Longo Prazo 🤖🧠

Neste notebook, vamos criar um agente que usa **memória persistente** para lembrar de interações passadas, utilizando um **banco vetorial** como mecanismo de armazenamento.

---

## 🟢 Objetivo

✅ Construir um **agente LLM** que armazena conhecimento em um **VectorStore**.  
✅ Consultar e atualizar a memória a cada interação, melhorando a coerência e a capacidade de lidar com histórico.

---

## 🛠️ Ferramentas

- **Python + Jupyter Notebook**  
- **LangChain** (integração de LLM + memory)  
- **ChromaDB** ou **FAISS** (armazenamento vetorial)  
- **OpenAI GPT-3.5/4** (motor de raciocínio)  
- **dotenv** para gerenciar API keys

---

## ⚙️ Etapa 1: Configuração inicial

In [1]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain

# Carregar variáveis de ambiente
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


## Inicializando o LLM e a memória vetorial

In [2]:
# Inicializando o LLM
llm = ChatOpenAI(api_key=openai_api_key, model_name="gpt-3.5-turbo", temperature=0)

# Inicializando embeddings para indexação
embeddings = OpenAIEmbeddings(api_key=openai_api_key)

# Criando o banco vetorial com Chroma
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_memory"  # pasta onde a memória será salva
)

# Conectando o retriever de memória
memory = VectorStoreRetrieverMemory(retriever=vectorstore.as_retriever())


C:\Users\vinicius\AppData\Local\Temp\ipykernel_11172\1189798483.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
C:\Users\vinicius\AppData\Local\Temp\ipykernel_11172\1189798483.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=vectorstore.as_retriever())


## Definindo o agente com memória

In [3]:
# Construindo a chain de conversa com memória vetorial
agent = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True  # exibe logs da interação
)

C:\Users\vinicius\AppData\Local\Temp\ipykernel_11172\668520311.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  agent = ConversationChain(


## Interagindo com o agente

In [4]:
# Primeira interação
resposta_1 = agent.invoke({"input": "Hello, I am testing your memory system!"})
print(resposta_1["response"])

# Segunda interação: o agente deve lembrar do contexto anterior
resposta_2 = agent.invoke({"input": "Do you remember what I just told you?"})
print(resposta_2["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, I am testing your memory system!
AI:

> Finished chain.
Hello! I'm ready for the test. Ask me anything you'd like.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Hello, I am testing your memory system!
response: Hello! I'm ready for the test. Ask me anything you'd like.
Human: Do you remember what I just told you?
AI:

> Finished chain.
Yes, you just mentioned that you are tes

##  Explorando o histórico de memória

In [5]:
# Checando o que está salvo na memória vetorial
docs = vectorstore.similarity_search("testing memory system")
for doc in docs:
    print(doc.page_content)


input: Hello, I am testing your memory system!
response: Hello! I'm ready for the test. Ask me anything you'd like.
input: Do you remember what I just told you?
response: Yes, you just mentioned that you are testing my memory system.


## Prova de fogo

In [10]:
# Inicializando o LLM
llm = ChatOpenAI(api_key=openai_api_key, model_name="gpt-3.5-turbo", temperature=0)

# Inicializando embeddings e banco vetorial
embeddings = OpenAIEmbeddings(api_key=openai_api_key)
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_memory")

# -----------------------------
# "Armazenando" o histórico manualmente no banco vetorial
# Simulando como se fossem anotações que o agente fez durante a conversa
vectorstore.add_texts(["Vamos falar sobre inteligência artificial."])

# -----------------------------
# AGENTE COM MEMÓRIA (faz busca explícita no banco vetorial)

print("\n🔹🔹 AGENTE COM MEMÓRIA 🔹🔹")

# 1️⃣ Falar de IA (salvando no Chroma)
primeira_interacao = "Vamos falar sobre inteligência artificial."
print(f"🟩 Entrada 1: {primeira_interacao}")
resposta_1 = llm.invoke(primeira_interacao)
print(f"🟩 Resposta 1: {resposta_1}")

# Simulando inserção no Chroma
vectorstore.add_texts([primeira_interacao])

# 2️⃣ Trocar de tópico
segunda_interacao = "Agora, vamos mudar para física quântica."
print(f"🟩 Entrada 2: {segunda_interacao}")
resposta_2 = llm.invoke(segunda_interacao)
print(f"🟩 Resposta 2: {resposta_2}")

# Simulando inserção no Chroma
vectorstore.add_texts([segunda_interacao])

# 3️⃣ Teste de memória
terceira_interacao = "Você lembra qual foi o primeiro tópico que conversamos?"
print(f"🟩 Entrada 3: {terceira_interacao}")

# Busca explícita no Chroma para injetar no prompt
retrieved_docs = vectorstore.similarity_search(terceira_interacao)
historico = "\n".join([doc.page_content for doc in retrieved_docs])

prompt_com_memoria = f"""
Histórico recuperado:
{historico}

Pergunta: {terceira_interacao}
"""

resposta_3 = llm.invoke(prompt_com_memoria)
print(f"🟩 Resposta 3 (com memória): {resposta_3}")

# -----------------------------
# AGENTE SEM MEMÓRIA (chamadas diretas ao LLM, sem injetar histórico)

print("\n🔹🔹 AGENTE SEM MEMÓRIA 🔹🔹")

# 1️⃣ Falar de IA
print(f"🟥 Entrada 1: {primeira_interacao}")
resposta_1_sem = llm.invoke(primeira_interacao)
print(f"🟥 Resposta 1: {resposta_1_sem}")

# 2️⃣ Trocar de tópico
print(f"🟥 Entrada 2: {segunda_interacao}")
resposta_2_sem = llm.invoke(segunda_interacao)
print(f"🟥 Resposta 2: {resposta_2_sem}")

# 3️⃣ Teste de memória (sem contexto recuperado)
print(f"🟥 Entrada 3: {terceira_interacao}")
resposta_3_sem = llm.invoke(terceira_interacao)
print(f"🟥 Resposta 3 (sem memória): {resposta_3_sem}")


🔹🔹 AGENTE COM MEMÓRIA 🔹🔹
🟩 Entrada 1: Vamos falar sobre inteligência artificial.
🟩 Resposta 1: content='Claro! A inteligência artificial é um campo da ciência da computação que se dedica a criar sistemas capazes de realizar tarefas que normalmente exigiriam inteligência humana, como reconhecimento de padrões, aprendizado, tomada de decisões e resolução de problemas. Esses sistemas são treinados com grandes quantidades de dados para aprender e melhorar seu desempenho ao longo do tempo.\n\nA inteligência artificial tem sido cada vez mais utilizada em diversas áreas, como na medicina, na indústria, no comércio, na segurança, entre outras. Ela pode ser aplicada em sistemas de recomendação, chatbots, carros autônomos, diagnósticos médicos, previsão de demanda, entre muitas outras aplicações.\n\nNo entanto, a inteligência artificial também levanta questões éticas e sociais, como o impacto no mercado de trabalho, a privacidade dos dados, o viés nos algoritmos e a responsabilidade dos sistema

In [11]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Carregar variáveis de ambiente
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Inicializando o LLM
llm = ChatOpenAI(api_key=openai_api_key, model_name="gpt-3.5-turbo", temperature=0)

# -----------------------------
# AGENTE COM MEMÓRIA AUTOMÁTICA
# Usando ConversationBufferMemory, que injeta automaticamente o histórico no prompt
memory_buffer = ConversationBufferMemory(return_messages=True)

agente_com_memoria = ConversationChain(
    llm=llm,
    memory=memory_buffer,
    verbose=True
)

# AGENTE SEM MEMÓRIA
# Não passa memória, cada interação isolada
agente_sem_memoria = ConversationChain(
    llm=llm,
    verbose=True
)

# -----------------------------
# Interações com AGENTE COM MEMÓRIA
print("\n🔹🔹 AGENTE COM MEMÓRIA 🔹🔹")

# 1️⃣ Falar de IA
entrada_1 = "Vamos falar sobre inteligência artificial."
print(f"🟩 Entrada 1: {entrada_1}")
resposta_1 = agente_com_memoria.invoke({"input": entrada_1})
print(f"🟩 Resposta 1: {resposta_1['response']}")

# 2️⃣ Trocar de tópico
entrada_2 = "Agora, vamos mudar para física quântica."
print(f"🟩 Entrada 2: {entrada_2}")
resposta_2 = agente_com_memoria.invoke({"input": entrada_2})
print(f"🟩 Resposta 2: {resposta_2['response']}")

# 3️⃣ Teste de memória
entrada_3 = "Você lembra qual foi o primeiro tópico que conversamos?"
print(f"🟩 Entrada 3: {entrada_3}")
resposta_3 = agente_com_memoria.invoke({"input": entrada_3})
print(f"🟩 Resposta 3 (com memória): {resposta_3['response']}")

# -----------------------------
# Interações com AGENTE SEM MEMÓRIA
print("\n🔹🔹 AGENTE SEM MEMÓRIA 🔹🔹")

# 1️⃣ Falar de IA
print(f"🟥 Entrada 1: {entrada_1}")
resposta_1_sem = agente_sem_memoria.invoke({"input": entrada_1})
print(f"🟥 Resposta 1: {resposta_1_sem['response']}")

# 2️⃣ Trocar de tópico
print(f"🟥 Entrada 2: {entrada_2}")
resposta_2_sem = agente_sem_memoria.invoke({"input": entrada_2})
print(f"🟥 Resposta 2: {resposta_2_sem['response']}")

# 3️⃣ Teste de memória (sem histórico!)
print(f"🟥 Entrada 3: {entrada_3}")
resposta_3_sem = agente_sem_memoria.invoke({"input": entrada_3})
print(f"🟥 Resposta 3 (sem memória): {resposta_3_sem['response']}")



🔹🔹 AGENTE COM MEMÓRIA 🔹🔹
🟩 Entrada 1: Vamos falar sobre inteligência artificial.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Vamos falar sobre inteligência artificial.
AI:

> Finished chain.
🟩 Resposta 1: Claro! A inteligência artificial é um campo da ciência da computação que se concentra no desenvolvimento de algoritmos e sistemas capazes de realizar tarefas que normalmente exigiriam inteligência humana. Isso inclui coisas como reconhecimento de padrões, tomada de decisões, processamento de linguagem natural e muito mais. Existem diferentes abordagens para a inteligência artificial, incluindo aprendizado de máquina, redes neurais e algoritmos de busca. É um campo fascinante e em constante evolução! 

### ✅ Quando não usamos ConversationChain (apenas .invoke() puro):

Cada chamada ao modelo é completamente isolada.

O LLM não tem nenhuma memória de contexto — só vê o prompt atual.

Então, em "Você lembra qual foi o primeiro tópico?", ele não sabe e vai dar uma resposta genérica (“Desculpe, não sei”).

### ✅ Quando usamos ConversationChain (mesmo sem explicitamente configurar memory=):

O LangChain automaticamente mantém um buffer de contexto de conversa:

Ele concatena as mensagens anteriores no prompt final que envia para o LLM.

Mesmo que você não passe uma ConversationBufferMemory explícita, o LangChain monta o prompt com as últimas interações.